### 1. Set environmental variables

In [1]:
from env_config import env_config

config = env_config()

2025-06-12 10:59:00,759 - INFO - Found a local .env file at /Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.env
2025-06-12 10:59:00,759 - INFO - Running in context: CLI
2025-06-12 10:59:00,760 - INFO - Force user authentication: False
2025-06-12 10:59:00,761 - INFO - Set balance of env values from.env file at /Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.env


OPTIONAL: Review environmental variables
WARNING: DO NOT COMMIT WITH THIS CELL OUTPUT SHOWING

In [ ]:
from pprint import pprint

pprint(config)

### 2. Init clients

In [2]:
from gcp_utils import get_gcp_credentials, init_sheets_client, init_drive_client
from qdrant_utils import init_qdrant_client


creds = get_gcp_credentials()
sheets_client = init_sheets_client(creds)
drive_client = init_drive_client(creds)
qdrant_client = init_qdrant_client("cloud")

2025-06-12 10:59:02,971 - INFO - Found a local .env file at /Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.env
2025-06-12 10:59:02,972 - INFO - Running in context: CLI
2025-06-12 10:59:02,972 - INFO - Force user authentication: False
2025-06-12 10:59:02,973 - INFO - Set balance of env values from.env file at /Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.env
2025-06-12 10:59:03,233 - INFO - Found a local .env file at /Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.env
2025-06-12 10:59:03,233 - INFO - Running in context: CLI
2025-06-12 10:59:03,233 - INFO - Force user authentication: False
2025-06-12 10:59:03,235 - INFO - Set balance of env values from.env file at /Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.env
2025-06-12 10:59:03,277 - INFO - ✅ Google Sheets client initialized successfully with scoped credentials.
2025-06-12 10:59:03,278 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-06-12 10:59:03,306 - INFO - ✅ Google Drive client

### 3. Test main orcehstration functions

Validate rows in LIBRARY_UNIFIED

In [3]:
from gcp_utils import fetch_sheet_as_df
from library_utils import validate_all_rows_format
from IPython.display import display, HTML


library_df = fetch_sheet_as_df(sheets_client, config["LIBRARY_UNIFIED"])
valid_df, invalid_df, log_df = validate_all_rows_format(library_df)

if not log_df.empty:
    print(f"\nValidation error detail:")
    display(HTML(log_df.to_html(notebook=True,
            max_cols=30, max_rows=10, escape=False)))

if not invalid_df.empty:
    print(f"\nInvalid rows detail:")
    display(HTML(invalid_df.to_html(notebook=True,
            max_cols=30, max_rows=10, escape=False)))

2025-06-12 10:59:07,192 - INFO - Sheet1
2025-06-12 10:59:07,424 - INFO - ✅ Fetched and converted worksheet 1glNjmKYloO0u6tCd5qy3z5ee8I-MkqwUxWsWEvLMBkM with 222 rows.
2025-06-12 10:59:07,440 - INFO - ✅ Found 221 valid rows in LIBRARY_UNIFIED.
2025-06-12 10:59:07,440 - INFO - ⚠️ Found 1 invalid rows in LIBRARY_UNIFIED.



Validation error detail:


,action,pdf_id,pdf_file_name,row_index,issues
0,validation_failed,d354afd1-95ea-5489-9df7-3abdd322d73c,CG Public Affairs CIM_5700_13A_2021-05-01.pdf,168,"missing_required_fields: ['google_id', 'link']"



Invalid rows detail:


,issues,title,pdf_id,publication_number,organization,scope,unit,issue_date,upsert_date,expiration_date,aux_specific,public_release,pdf_file_name,embedding,google_id,link,status,status_timestamp
166,"missing_required_fields: ['google_id', 'link']","COAST GUARD EXTERNAL AFFAIRS MANUAL, COMDTINST...",d354afd1-95ea-5489-9df7-3abdd322d73c,COMDTINST M5700.13A,CG-092,National,,2021-05-12T00:00:00Z,,2031-05-13T00:00:00Z,FALSE,TRUE,CG Public Affairs CIM_5700_13A_2021-05-01.pdf,text-embedding-ada-002,,,orphan_row,


Promote tagged

In [ ]:
from promote_files import promote_files

promote_files(drive_client, sheets_client, qdrant_client)

Delete tagged


In [4]:
from delete_tagged import delete_tagged


deleted_rows_df = delete_tagged(drive_client, sheets_client, qdrant_client)

2025-06-12 10:59:33,096 - INFO - Found a local .env file at /Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.env
2025-06-12 10:59:33,096 - INFO - Running in context: CLI
2025-06-12 10:59:33,096 - INFO - Force user authentication: False
2025-06-12 10:59:33,098 - INFO - Set balance of env values from.env file at /Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.env
2025-06-12 10:59:33,099 - INFO - Found a local .env file at /Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.env
2025-06-12 10:59:33,099 - INFO - Running in context: CLI
2025-06-12 10:59:33,100 - INFO - Force user authentication: False
2025-06-12 10:59:33,101 - INFO - Set balance of env values from.env file at /Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.env
2025-06-12 10:59:33,673 - INFO - Sheet1
2025-06-12 10:59:33,846 - INFO - ✅ Fetched and converted worksheet 1glNjmKYloO0u6tCd5qy3z5ee8I-MkqwUxWsWEvLMBkM with 222 rows.
2025-06-12 10:59:33,848 - INFO - Found 0 rows matching keywords: ['deletion']
2

cleanup orphans

In [5]:
from cleanup_orphans import find_orphans
from IPython.display import display, HTML


orphan_rows, orphan_files, orphan_qdrant_records, log_df = find_orphans(
    drive_client, sheets_client, qdrant_client)

print(f"\nOrphan rows:")
if not orphan_rows.empty:
    display(HTML(orphan_rows.to_html(notebook=True,
            max_cols=30, max_rows=10, escape=False)))

print(f"\nOrphan files:")
if not orphan_files.empty:
    display(HTML(orphan_files.to_html(notebook=True,
            max_cols=30, max_rows=10, escape=False)))

print(f"\nOrphan qdrant records:")
if not orphan_qdrant_records.empty:
    display(HTML(orphan_qdrant_records.to_html(notebook=True,
            max_cols=30, max_rows=10, escape=False)))

2025-06-12 10:59:39,503 - INFO - Found a local .env file at /Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.env
2025-06-12 10:59:39,503 - INFO - Running in context: CLI
2025-06-12 10:59:39,503 - INFO - Force user authentication: False
2025-06-12 10:59:39,504 - INFO - Set balance of env values from.env file at /Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.env
2025-06-12 10:59:40,203 - INFO - Sheet1
2025-06-12 10:59:40,431 - INFO - ✅ Fetched and converted worksheet 1glNjmKYloO0u6tCd5qy3z5ee8I-MkqwUxWsWEvLMBkM with 222 rows.
2025-06-12 10:59:40,432 - INFO - Cataloging PDF_TAGGING for orphan review
2025-06-12 10:59:41,123 - INFO - Cataloging PDF_LIVE for orphan review
2025-06-12 10:59:44,131 - INFO - Sheet1
2025-06-12 10:59:44,133 - INFO - orphan_row_flagged in LIBRARY_UNIFIED — pdf_id: d354afd1-95ea-5489-9df7-3abdd322d73c, file: CG Public Affairs CIM_5700_13A_2021-05-01.pdf
2025-06-12 10:59:44,440 - INFO - ✅ Updated 1 orphan rows in LIBRARY_UNIFIED.
2025-06-12 10:59:44,442


Orphan rows:


,title,pdf_id,publication_number,organization,scope,unit,issue_date,upsert_date,expiration_date,aux_specific,public_release,pdf_file_name,embedding,google_id,link,status,status_timestamp
166,"COAST GUARD EXTERNAL AFFAIRS MANUAL, COMDTINST...",d354afd1-95ea-5489-9df7-3abdd322d73c,COMDTINST M5700.13A,CG-092,National,,2021-05-12T00:00:00Z,,2031-05-13T00:00:00Z,FALSE,TRUE,CG Public Affairs CIM_5700_13A_2021-05-01.pdf,text-embedding-ada-002,,,orphan_row,



Orphan files:


,ID,Name,URL,folder
14,1R6DtX_1jTKf3WEDARSmW2Ad7WbbxuClG,2024-2026_Strategic_Plan_-_November_2024.pdf,https://drive.google.com/file/d/1R6DtX_1jTKf3W...,PDF_LIVE
38,1C_VU-mFfnEovVaSFeqz0EhDvcPMvqjwa,10_15_AUXILIARY_MANDATED_TRAINING__AUXMT____NE...,https://drive.google.com/file/d/1C_VU-mFfnEovV...,PDF_LIVE
39,1xO12dBYJBxVP7XBqBIRpYYN4dHynmRhh,ALAUX_002_24_2024_NATIONAL_WORKSHOPS.pdf,https://drive.google.com/file/d/1xO12dBYJBxVP7...,PDF_LIVE
41,19Ql6ALvx6LEGlYTdOgWTEzosc5LnWCXq,06_19_AUXILIARY_HUMANITARIAN_SERVICE_AWARD.pdf,https://drive.google.com/file/d/19Ql6ALvx6LEGl...,PDF_LIVE
57,15CV77p3T5BQPOhHz1vdIEbA82KIEYEDM,VE_workshop_Dec_4_22_2023.pdf,https://drive.google.com/file/d/15CV77p3T5BQPO...,PDF_LIVE
...,...,...,...,...
136,1m8aKoPaThcwAK4dfFaJGkaI6JxA1ngxK,Surface Workshop Final 2024 v1.pdf,https://drive.google.com/file/d/1m8aKoPaThcwAK...,PDF_LIVE
166,17Bevh6ZUi2L0vz0xSfL947_KRpQY3kpz,AUXFS Training and Qualification Guide_H-Dir_2...,https://drive.google.com/file/d/17Bevh6ZUi2L0v...,PDF_LIVE
167,1X_EltW4CzIG5ZxxZAf866GOKU_EySqKe,12_15_TEMPORARY_SUSPENSION_OF_UDC_Istore___12J...,https://drive.google.com/file/d/1X_EltW4CzIG5Z...,PDF_LIVE
183,1CTodsNNKKvVzVvzIDkgpA-feqwe65A9W,IT Instructor WORKSHOP 2023 Jan 2023 Final.pdf,https://drive.google.com/file/d/1CTodsNNKKvVzV...,PDF_LIVE



Orphan qdrant records:


,pdf_id,title,pdf_file_name,page_count,record_count,point_ids
0,d354afd1-95ea-5489-9df7-3abdd322d73c,"COAST GUARD EXTERNAL AFFAIRS MANUAL, COMDTINST...",None,133,132,"[00a80758-8e6b-43ae-8208-296ae0af01f6, 021434b..."


### 4. Test helper functions

In [ ]:
pdf_ids = [
    "f43bdf6d-07ac-5ce0-b206-db7eabb990ca",
    "5098752e-4166-55f2-9f46-227901cdfc9c",
    "1ff90293-9f4e-568a-a3ac-1a2e9a01838d",
    "b4b536dc-7acc-5ac4-a200-e92bfc3447a3",
    "712095dd-af64-5522-8d96-ce745c700fc2",
    "8c9e8a61-0c9c-5650-9020-ce6a07624716",
    "a23943b3-c50d-5a0e-8094-4782d5022e25",
    "08b3baae-3aa2-5ccd-8c80-8ab255a45156",
    "f7b993d6-1ce8-541b-81d6-0db6076062ca",
    "959f3f11-e812-5b59-b2f1-07f76a9025ef",
    "d10522f0-1909-55e2-b03c-6eef68f39bef",
    "41363a2d-0a9c-5e19-9b57-f85cb0e8f3fe",
    "12abc1d1-bec1-58b2-8763-7158f77e8698",
    "2d4143fd-7432-577e-8f5f-ac482185d292",
    "d354afd1-95ea-5489-9df7-3abdd322d73c",
    "09544186-4593-5579-b2af-09fb953b7e86",
    "4b7c9994-8f65-5a1d-8e8a-7d73f4b38de8",
    "21126abd-5489-5ffa-b566-d6f365d5a74b",
    "70e0849b-8f89-5242-9a40-c10b88173ac1",
    "2d6a042f-0c21-54b6-bd4f-588b8be73999",
    "75fddd31-e90f-5311-a38c-6d6f807d1fdc",
    "52d35e10-7df5-5ab8-b2a0-110c26c6d2b2",
    "3b845a10-cb3a-5014-96d8-360c8f1bf63f",
    "ba4883c2-9545-5d70-868a-867eea0a7678",
    "e214b5b8-9077-555d-b101-0b7eb5fef90a",
    "9edb0da8-7257-59d4-9b27-5bcede517047",
    "4527cfc9-f47d-5945-8e0d-9e98e1fce929",
    "c15f7a15-6f60-5064-870c-123a26d86bc6",
    "cc43c3f2-f44f-5df0-aab8-3f50359b0292",
    "9e8faabf-0fbd-5bcf-b7e1-00b859360f17",
    "bc8c6a12-932a-51ca-b40d-13fb1e6ab0a9",
    "c7143b60-c938-5329-9a3e-4d3411a2827b",
    "c820d16d-b010-5ca1-bd76-f877dc987c93",
    "23746b1e-b829-55fc-8777-8c85ec3a9f41",
    "7b5a850a-b5b2-50cf-bc87-1d5a0c92b322",
    "149e15a5-8f4f-53a6-a3dc-1becb28560d4",
    "d2c55494-6863-5186-a423-e83e32fad3b3",
    "042ba9b7-ee51-541f-9ec5-71ed26104824",
    "29d16974-c3a5-5c89-8997-262037815c93",
    "25e956d7-3dd1-5126-b9b9-2407e53e2121"
]

In [ ]:
from IPython.display import display, HTML
from env_config import rag_config
from qdrant_utils import get_summaries_by_pdf_id


summaries_df = get_summaries_by_pdf_id(
    qdrant_client, rag_config("qdrant_collection_name"), pdf_ids)

if not summaries_df.empty:
    display(HTML(summaries_df.to_html(notebook=True,
            max_cols=30, max_rows=10, escape=False)))

In [6]:
from gcp_utils import fetch_sheet

sheet = fetch_sheet(sheets_client, config["LIBRARY_UNIFIED"])
cell_value = sheet.cell(2, 3).value  # Row 2, Column 3
print(cell_value)

2025-06-12 11:00:09,340 - INFO - Sheet1


ALAUX_001_23


In [7]:
from library_utils import fetch_rows_by_status, remove_rows
from gcp_utils import get_folder_name, fetch_sheet_as_df, file_exists
import pandas as pd

library_df = fetch_sheet_as_df(sheets_client, config["LIBRARY_UNIFIED"])
display(library_df.head())

2025-06-12 11:01:18,681 - INFO - Sheet1
2025-06-12 11:01:18,865 - INFO - ✅ Fetched and converted worksheet 1glNjmKYloO0u6tCd5qy3z5ee8I-MkqwUxWsWEvLMBkM with 222 rows.


,title,pdf_id,publication_number,organization,scope,unit,issue_date,upsert_date,expiration_date,aux_specific,public_release,pdf_file_name,embedding,google_id,link,status,status_timestamp
0,ALAUX 001/23 EXTENSION TO AUX SOP 004A 04 Sep ...,89328d34-2b70-507e-b7ba-889ee6cf3024,ALAUX_001_23,CG-BSX,National,,2023-01-12T00:00:00Z,,2033-01-12T00:00:00Z,TRUE,TRUE,001_23_EXTENSION_TO_AUX_SOP_004A_04_Sep_20__EL...,text-embedding-ada-002,1Usl8gU9VM8jX8ODj9EHH_rE4QJuH3tQ5,https://drive.google.com/file/d/1Usl8gU9VM8jX8...,live,
1,ALAUX 003/24 UPDATE RISK MANAGEMENT TRAINING R...,0309c2d2-9ffc-5f95-9dea-68515ff270c9,ALAUX_003_24,CG-BSX,National,,2024-01-30T00:00:00Z,,2034-01-29T00:00:00Z,TRUE,TRUE,003_24_UPDATE_RISK_MANAGEMENT_TRAINING_REQUIRE...,text-embedding-ada-002,1U_ogBXiH842fxEOkW2lCeY5pkgHR_g00,https://drive.google.com/file/d/1U_ogBXiH842fx...,live,
2,ALAUX 004/23 AUXILIARY COVID 19 GUIDANCE VERSI...,a603f875-b49c-5c72-bd01-40401bef19a2,ALAUX_004_23,CG-BSX,National,,2023-01-20T00:00:00Z,,2033-01-20T00:00:00Z,TRUE,TRUE,004_23_AUXILIARY_COVID_19_GUIDANCE_VERSION_5.pdf,text-embedding-ada-002,1ArwnK66rE2O6d5wxxIbjH8Ns1eeijTKG,https://drive.google.com/file/d/1ArwnK66rE2O6d...,live,
3,ALAUX 005/23 BOATING SAFETY APPLICATION FOR SM...,143b6af4-fbf4-5c48-8c71-a685e3e655eb,ALAUX_005_23,CG-BSX,National,,2023-01-27T00:00:00Z,,2033-01-26T00:00:00Z,TRUE,TRUE,005_23_BOATING_SAFETY_APPLICATION_FOR_SMART_PH...,text-embedding-ada-002,13bdh7S3HqVB5H-DSpgZ5CSlXneJmf9UF,https://drive.google.com/file/d/13bdh7S3HqVB5H...,live,
4,ALAUX 006/13 CAMPAIGN TO ELIMINATE SEXUAL ASSA...,b15229c0-03ca-5ba3-8466-9c06c0535493,ALAUX_006_13,CG-BSX,National,,2019-11-12T00:00:00Z,,2029-11-12T00:00:00Z,TRUE,TRUE,006_13_CAMPAIGN_TO_ELIMINATE_SEXUAL_ASSAULT___...,text-embedding-ada-002,1NUoA1enlRc6FnvGzAIhb7CYGuxlE0oY3,https://drive.google.com/file/d/1NUoA1enlRc6Fn...,live,
